In [13]:
import os
import pickle
import time

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait

brower = webdriver.Chrome()
wait = WebDriverWait(brower, 10)

def getTaobaoCookies():
    # get login taobao cookies
    url = "https://s.taobao.com/search?q=ipad"
    brower.get("https://s.taobao.com/search?q=ipad")
    while True:
        print("Please login in taobao.com!")
        time.sleep(3)
        # if login in successfully, url  jump to www.taobao.com
        while brower.current_url ==  url:
            tbCookies  = brower.get_cookies()
            brower.quit()
            cookies = {}
            for item in tbCookies:
                cookies[item['name']] = item['value']
            outputPath = open('taobaoCookies.pickle','wb')
            pickle.dump(cookies,outputPath)
            outputPath.close()
            return cookies


In [14]:
getTaobaoCookies()

Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!
Please login in taobao.com!


{'JSESSIONID': '97D5C7F7AEE823602D50C70CB715A447',
 '_cc_': 'VT5L2FSpdA%3D%3D',
 '_l_g_': 'Ug%3D%3D',
 '_nk_': '%5Cu51E1%5Cu51E1%5Cu51E1%5Cu51E1%5Cu54E5%5Cu54E5',
 '_tb_token_': '5ebede33bd3b7',
 'cna': '22k1Fmq/OBoCAXeJNvlseDBc',
 'cookie1': 'UtRWWKdRjVunVM0qv625ErElZZ57upx0n3D3EtCeY4s%3D',
 'cookie17': 'UUpkv6oAkw%2BH3A%3D%3D',
 'cookie2': '1744077528795ba46a2ef3d902c2da53',
 'csg': 'd4699c87',
 'dnk': '%5Cu51E1%5Cu51E1%5Cu51E1%5Cu51E1%5Cu54E5%5Cu54E5',
 'enc': 'P%2BWz%2BvkuqpDNuCPXyd3yfTWnMWLbrMnbxv5du1FCZXcchKi9VYFET50TAEYVdc9TFPYcQCOzk7ydS7YzkGnLRw%3D%3D',
 'existShop': 'MTU3MTcxNjA5Mw%3D%3D',
 'hng': 'CN%7Czh-CN%7CCNY%7C156',
 'isg': 'BI-P0rayDigWxwrU6hdH44EMHiVZHOPYLijeCqGcK_4FcK9yqYRzJo3iduHrE7tO',
 'l': 'dBSGjCi4qklidRpzBOCanurza77OSIRYYuPzaNbMi_5Cr6T6zIQOkMJ7uF96VjX5GBLBqDlUQKv9-etbZAo6E4K-g3frwHASNFxX4',
 'lgc': '%5Cu51E1%5Cu51E1%5Cu51E1%5Cu51E1%5Cu54E5%5Cu54E5',
 'mt': 'ci=21_1',
 'sg': '%E5%93%A595',
 'skt': '637f8984f8cbd0c9',
 't': 'eacf485faac48a3f8302b0a7f37ffe53',
 't

In [23]:
def readTaobaoCookies():
    # if hava cookies file ,use it 
    # if not , getTaobaoCookies()
    if os.path.exists('taobaoCookies.pickle'):
        readPath = open('taobaoCookies.pickle','rb')
        tbCookies = pickle.load(readPath)
    else:
        tbCookies = getTaobaoCookies()
    return tbCookies

In [45]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from urllib.parse import quote
from pyquery import PyQuery as pq

browser = webdriver.Chrome()
wait = WebDriverWait(browser,5)
tbCookies = readTaobaoCookies()

browser.get("https://www.taobao.com")
for cookie in tbCookies:
    browser.add_cookie({
        "domain":".taobao.com",
        "name":cookie,
        "value":tbCookies[cookie],
        "path":'/',
        "expires":None
    })
KEYWORD = 'ipad'

def index_Page(page):
    print("page:",page)
    try:
        url = 'https://s.taobao.com/search?q='+quote(KEYWORD)
        browser.get(url)
        if page>1:
            input = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR,'#mainsrp-pager div.form > input'))
            )
            submit = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR,'#mainsrp-pager div.form > span.btn.J_Submit'))
                #EC.presence_of_element_located((By.CSS_SELECTOR,'#mainsrp-page div.form > span.J_Submit'))
            )
            input.clear()
            input.send_keys(page)
            submit.click()
        wait.until(
            EC.text_to_be_present_in_element((By.CSS_SELECTOR,'#mainsrp-pager li.item.active > span'),str(page))
        )
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,'.m-itemlist .items .item')))
        get_products()
    except TimeoutException:
        index_Page(page)

def get_products():
    html = browser.page_source
    doc = pq(html)
    items = doc('#mainsrp-itemlist .items .item').items()
    products = []
    for item in items:
        product = {
            'image': item.find('.pic .img').attr('data-src'),
            'price': item.find('.price').text(),
            'deal': item.find('deal-cnt').text()
        }
        products.append(product)
    print(products)


MAX_PAGE = 100
def main():
    for i in range(1,MAX_PAGE+1):
        index_Page(i)

main()

page: 1
[{'deal': '', 'price': '¥2699.00', 'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i3/1917047079/O1CN01LRD0NC22AEJDhQixD_!!2-item_pic.png'}, {'deal': '', 'price': '¥2921.00', 'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i2/1917047079/O1CN01orvMCo22AEFyGqSrB_!!0-item_pic.jpg'}, {'deal': '', 'price': '¥4870.00', 'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i1/1917047079/O1CN01qNZUmy22AEFzBvXlR_!!2-item_pic.png'}, {'deal': '', 'price': '¥2298.00', 'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i2/1776456424/O1CN01puCczr1xKEtNs3wX1_!!0-item_pic.jpg'}, {'deal': '', 'price': '¥2499.00', 'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i1/1669409267/O1CN01j5c1bq2IKKuO3Zc0F_!!1669409267-0-pixelsss.jpg'}, {'deal': '', 'price': '¥3896.00', 'image': '//g-search2.alicdn.com/img/bao/uploaded/i4/i1/2616970884/O1CN01F46RVW1IOueSf2PgD_!!0-item_pic.jpg'}, {'deal': '', 'price': '¥2499.00', 'image': '//g-search3.alicdn.com/img/bao/uploaded/i4/i1/1776456424/O1CN01Yrm

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=77.0.3865.120)
